In [23]:
from numba import jit, cuda 
from scipy.signal import convolve2d
from scipy.integrate import ode
import numpy as np
from PIL import Image as img

In [26]:
# hàm trạng thái
def hamTrangThai(t, x, u, quanThe, n, m):
    A = taoMaTranDoiXungPerceptron(quanThe[:5])
    B = taoMaTranDoiXungPerceptron(quanThe[5:10])
    I = quanThe[10]
    x = x.reshape(n, m)
    dx = convolve2d(u, B, 'same') + convolve2d(hamDauRa(x), A, 'same') + I
    return dx.reshape(n*m)

# hàm đầu ra
def hamDauRa(x):
    return 0.5 * (np.abs(x + 1) - np.abs(x - 1))

# Tạo ma trận đối xứng từ 5 trọng số (hàm này chỉ dành riêng cho ma trận đối xứng 3x3)
def taoMaTranDoiXung(arr1):#giả sử cho mảng [1,2,3,4,5]
    arr2 = arr1[-2::-1].copy()#tạo mảng phụ [4,3,2,1]
    #arr2[0], arr2[2] = arr2[2], arr2[0]#nếu có thì sẽ đối xứng theo đường chéo phụ, nếu không thì sẽ là đối xứng tâm
    return np.append(arr1,arr2).reshape(h,h)#kết hợp hai mảng rồi chuyển về dạng ma trận 3x3

# Trong công thức biến đổi từ mạng Nơ-ron phẩn hồi sang truyền thẳng, trọng số và đầu vào , đầu ra phải được nhân với nhau theo thứ tự nhất định (xem lại công thứ)
# hàm sau đây sẽ sắp xếp lại các số trọng bộ ma trận trọng số để tương ứng với công thức.
def taoMaTranDoiXungPerceptron(arr1):
    return taoMaTranDoiXung(np.array([arr1[2],arr1[1],arr1[3],arr1[0],arr1[4]]))

# Hàm tạo số ngẫu nhiên từ -9.99 đến 9.99
def ngauNhien():
    return np.random.randint(-999,1000)/100.

# Hàm tính sai số
def saiSo(d,y):
    return np.sum(0.5*(d-y)**2)

def loadSample(path):
    return np.array(img.open(path).convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0

# Bán kính lân cận của nơ ron tế bào
r = 1

# kích thước của ma trận điều khiển và ma trận phản hồi (hai ma trận này luôn vuông)
h = 2*r + 1 # h = 3

class CNN:

    slts = 11 # số lượng trọng số trong quần thể

    # Quần thể trọng số (gồm 11 số)
    # quanThe = np.array([ngauNhien() for i in range(slts)])
    quanThe = np.array([0.,0.,0.,0.,2.,0.,0.,0.,0.,0.,-1.])

    # ma trận ảnh đầu vào u
    # u = np.array(img.open('images/u1_1.png').convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0
    u = [
        loadSample('images/u1_1.png'),
        loadSample('images/u1_2.png'),
        loadSample('images/u1_3.png'),
        loadSample('images/u1_4.png'),
        loadSample('images/u1_5.png'),
    ]

    # kích thước ma trận đầu vào, ma trận đầu ra và ma trận trạng thái
    m, n = u[0].shape
    print(m,n)

    # ma trận đầu ra mong muốn d
    # d = np.array(img.open('images/d1_1.png').convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0
    d = [
        loadSample('images/d1_1.png'),
        loadSample('images/d1_2.png'),
        loadSample('images/d1_3.png'),
        loadSample('images/d1_4.png'),
        loadSample('images/d1_5.png'),
    ]

    # hai giá trị ban đầu cho ptvp gồm x0 = [ma trận toàn số 0] với t0 = 0.0 và t1 để tính đầu ra x1 (t1 != t0)
    x0, t0, t1 = [i.copy() for i in u], 0.0, 0.05

    # ma trận đầu ra y (y được tính theo x)
    y = [hamDauRa(i) for i in x0]

    # Tốc độ học của thuật toán
    alpha = 0.005

    E0 = 6.

    E = [1000]*len(u)

cnn = CNN()

# In ra bộ ma trận trọng số:
def print_A_B_I():
    print(f'A = {taoMaTranDoiXung(cnn.quanThe[:5])}')
    print(f'B = {taoMaTranDoiXung(cnn.quanThe[5:10])}')
    print(f'I = {cnn.quanThe[10]}')

# Hàm để lấy trọng số từ file trongSoBac1ToiUu rồi lưu vào cnn.quanThe
def getQuanThe(i):# lấy bộ trọng số thứ i
    A = np.loadtxt(f'TrongSoBac1ToiUu/TrongSo{i}/A.txt')
    B = np.loadtxt(f'TrongSoBac1ToiUu/TrongSo{i}/B.txt')
    I = np.loadtxt(f'TrongSoBac1ToiUu/TrongSo{i}/I.txt')
    cnn.quanThe = np.array([A[0][0],A[0][1],A[0][2],A[1][0],A[1][1],B[0][0],B[0][1],B[0][2],B[1][0],B[1][1],I])

# tạo thêm biên gồm toàn số 0 bao quanh ma trận a để tránh tràn biên
def them_bien(a):
    b = np.zeros(shape=[a.shape[0]+2,a.shape[1]+2])
    for i in range(1,a.shape[0]+1):
        for j in range(1,a.shape[1]+1):
            b[i][j] = a[i-1][j-1]
    return b

# tính x, tính y, tính wij và cập trọng số
#@jit(target_backend='cuda')
def capNhatTrongSo():
    for t in range(len(cnn.u)):
        y = them_bien(cnn.y[t]) # thêm biên 0 cho y để tránh tràn biên
        u = them_bien(cnn.u[t]) # thêm biên 0 cho u để tránh tràn biên
        for i in range(1,cnn.m+1):
            for j in range(1,cnn.n+1):
                Ys = np.array([
                    y[i-1][j] + y[i+1][j], 
                    y[i][j-1] + y[i][j+1],
                    y[i-1][j-1] + y[i+1][j+1],
                    y[i-1][j+1] + y[i+1][j-1],
                    y[i][j],
                    u[i-1][j] + u[i+1][j], 
                    u[i][j-1] + u[i][j+1],
                    u[i-1][j-1] + u[i+1][j+1],
                    u[i-1][j+1] + u[i+1][j-1],
                    u[i][j],
                    1            
                ])
                norm_n = np.sum(Ys**2)
                wij = cnn.alpha * (cnn.d[t][i-1][j-1] - y[i][j]) / norm_n
                if(wij != 0.):
                    for ts in range(cnn.slts):
                        cnn.quanThe[ts] += wij * Ys[ts]
        # tính toán lại sai số
        ptvp[t].set_initial_value(cnn.x0[t].flatten(), 0.)
        ptvp[t].set_f_params(cnn.u[t], cnn.quanThe, cnn.n, cnn.m)
        assert ptvp[t].successful()
        x = (ptvp[t].integrate(cnn.t1))[:].reshape(cnn.n, cnn.m)
        cnn.y[t] = np.sign(hamDauRa(x))

ptvp = [ode(hamTrangThai).set_integrator('vode')]*len(cnn.u) # thuật toán được sử dụng để giải ptvp[i], đọc thêm trong tài liệu

for i in range(len(cnn.u)):
    ptvp[i].set_initial_value(cnn.x0[i].flatten(), 0.) # lệnh flatten dùng để biến đổi ma trận về dạng mảng (đầu vào của ode phải là một số hoặc mảng)

    print_A_B_I()
    print(f'u = {cnn.u[i]}')
    print(f'd = {cnn.d[i]}')

    ptvp[i].set_f_params(cnn.u[i], cnn.quanThe, cnn.n, cnn.m)
    assert ptvp[i].successful()
    x = (ptvp[i].integrate(cnn.t1))[:].reshape(cnn.n, cnn.m)
    y = hamDauRa(x)
    cnn.y[i] = np.sign(y)
    print(f'y = {cnn.y[i]}')
    cnn.E[i] = saiSo(cnn.d[i],cnn.y[i])
    print(f'Sai số: E = {cnn.E[i]}')

8 8
A = [[0. 0. 0.]
 [0. 2. 0.]
 [0. 0. 0.]]
B = [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
I = -1.0
u = [[ 1.  1.  1. -1.  1.  1. -1. -1.]
 [ 1.  1.  1. -1.  1.  1. -1. -1.]
 [ 1.  1. -1. -1. -1.  1.  1. -1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1. -1. -1. -1. -1. -1.  1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]]
d = [[ 1.  1.  1. -1.  1.  1. -1. -1.]
 [ 1.  1.  1. -1.  1.  1. -1. -1.]
 [ 1.  1. -1.  1. -1.  1.  1. -1.]
 [ 1.  1. -1.  1. -1.  1.  1.  1.]
 [ 1. -1.  1.  1.  1. -1.  1.  1.]
 [ 1. -1.  1.  1.  1. -1.  1.  1.]
 [-1.  1.  1.  1.  1.  1. -1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]]
y = [[ 1.  1.  1. -1.  1.  1. -1. -1.]
 [ 1.  1.  1. -1.  1.  1. -1. -1.]
 [ 1.  1. -1. -1. -1.  1.  1. -1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1. -1. -1. -1. -1. -1.  1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]]
Sai số: E = 26.0
A = [[0. 0. 0.]
 [0. 2. 0.]
 [0. 

In [27]:
print('Bộ trọng số ban đầu: ')
print(f'Tổng sai số ban đầu: {sum(cnn.E)}')
# vòng lặp lớn
loop = 10000 #số lần chạy thử tối đa
count = 0 # đếm số lần chạy
while(True):
    # kiểm tra ptvp ODE
    if([i.successful() for i in ptvp].count(False)):
        print('Lỗi phương trình vi phân')
        break

    # đếm số lần chạy
    if(count>loop):
        print('Đạt đến số vòng lặp tối đa')
        break
    else:
        print(f'Lần chạy thứ {count+1}')
        
    capNhatTrongSo()

    count+=1

    cnn.E = [saiSo(cnn.d[i],cnn.y[i]) for i in range(len(cnn.u))]
    if(max(cnn.E) <= cnn.E0):
        print('Đạt đến mức sai số cho phép')
        break

    # print_maTranTrongSo()
    # print(f'Tốc độ học: {cnn.alpha}')
    print(f'Sai số: {cnn.E}. Sai số max: {max(cnn.E)}')

print('Bộ trọng sau khi tối ưu: ')
print(f'Sai số: {cnn.E}. Sai số max: {max(cnn.E)}')

Bộ trọng số ban đầu: 
Tổng sai số ban đầu: 192.0
Lần chạy thứ 1
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 2
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 3
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 4
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 5
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 6
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 7
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 8
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 9
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 10
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 11
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 12
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy thứ 13
Sai số: [26.0, 20.0, 24.0, 52.0, 70.0]. Sai số max: 70.0
Lần chạy th

In [30]:
#test với ảnh mẫu
listImage = [1,2,6,7]
fileExtension = ['bmp','gif','jpg','png']
# listImage = [4]
# fileExtension = ['jpg']

# maTranAnh = [
#     img.open('images/input1.bmp').convert('RGB'),
#     img.open('images/input2.gif').convert('RGB'),
#     img.open('images/input6.jpg').convert('RGB'),
#     img.open('images/input7.png').convert('RGB'),
#     img.open('images/input4.jpg').convert('RGB'),
#     ]

for i in range(len(listImage)):
    maTranAnh = img.open(f'images/input{listImage[i]}.{fileExtension[i]}').convert('RGB')
    m,n = maTranAnh.size
    u = np.array(maTranAnh)[:,:,0]
    print(u.shape)
    u = u/255.0*2.0-1.0
    ptvp_test = ode(hamTrangThai).set_integrator('vode').set_initial_value(u.copy().flatten(), 0.0)
    ptvp_test.set_f_params(u, cnn.quanThe, n, m)
    assert ptvp_test.successful()
    x = (ptvp_test.integrate(cnn.t1))[:].reshape(n, m)
    y = hamDauRa(x)
    y = np.sign(y)
    y = (y/2.0+0.5)*255
    print(f'y = {y}')
    out = img.fromarray(y).convert('RGB')
    out.save(f'images/output{listImage[i]}.png')
    print(f'done image {listImage[i]}')
    # maTranAnh.show()
    # out.show()

(256, 256)
y = [[255. 255. 255. ... 255. 255. 255.]
 [255.   0.   0. ...   0.   0. 255.]
 [255.   0.   0. ...   0.   0. 255.]
 ...
 [255.   0.   0. ...   0.   0. 255.]
 [255.   0.   0. ...   0.   0. 255.]
 [255. 255. 255. ... 255. 255. 255.]]
done image 1
(128, 128)
y = [[255.   0.   0. ... 255. 255. 255.]
 [255. 255.   0. ... 255.   0. 255.]
 [  0. 255. 255. ... 255.   0. 255.]
 ...
 [  0. 255. 255. ... 255. 255.   0.]
 [  0. 255. 255. ... 255. 255.   0.]
 [255.   0.   0. ...   0.   0.   0.]]
done image 2
(252, 336)
y = [[255. 255. 255. ... 255. 255. 255.]
 [255.   0.   0. ...   0.   0. 255.]
 [255.   0.   0. ...   0.   0. 255.]
 ...
 [255.   0.   0. ...   0.   0. 255.]
 [255.   0.   0. ...   0.   0. 255.]
 [255. 255. 255. ... 255. 255. 255.]]
done image 6
(316, 474)
y = [[255. 255. 255. ... 255. 255. 255.]
 [255.   0.   0. ...   0.   0. 255.]
 [255.   0.   0. ...   0.   0. 255.]
 ...
 [  0. 255. 255. ...   0.   0. 255.]
 [  0. 255. 255. ...   0.   0. 255.]
 [  0.   0.   0. ... 255. 2